In [1]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import nolds

from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
from mne.channels import make_standard_montage


from scipy.signal import detrend
from tqdm import tqdm
from scipy.stats import linregress

from mne.time_frequency import tfr_array_morlet
from scipy.signal import hilbert, butter, filtfilt


In [2]:
import my_functions2 as mf

In [3]:
subject = 1
runs = {
    "rest": [1],
    "motor_execution": [3, 7, 11],
    "motor_imagery": [4, 8, 12]
}

# Load and preprocess the data
subject_data = mf.load_and_preprocess_subject(subject, runs)


➡️ Loading REST | Runs: [1]
Extracting EDF parameters from C:\Users\flavi\mne_data\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cu

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 529 samples (3.306 s)

✅ motor_execution | Shape: (64, 60000) | Duration: 6.25 min

➡️ Loading MOTOR_IMAGERY | Runs: [4, 8, 12]
Extracting EDF parameters from C:\Users\flavi\mne_data\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R04.ed

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\flavi\mne_data\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition 

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


In [ ]:
%matplotlib qt

# Quick visualization check
mf.quick_plot(subject_data["rest"], title=f"Subject {subject} | REST")
mf.quick_plot(subject_data["motor_execution"], title=f"Subject {subject} | MOTOR EXECUTION")

In [ ]:
# Extract epochs from each condition
epochs_rest = mf.extract_clean_epochs(subject_data["rest"])
epochs_exec = mf.extract_clean_epochs(subject_data["motor_execution"]) 
epochs_imag = mf.extract_clean_epochs(subject_data["motor_imagery"])

In [ ]:
# Combine epochs for more robust analysis
combined_rest = mne.concatenate_epochs([
    epochs_exec['rest'], 
    epochs_imag['rest'], 
    epochs_rest['rest']
])

combined_task = mne.concatenate_epochs([
    epochs_exec['task'], 
    epochs_imag['task']
])

# Shuffle task epochs for fair sampling
np.random.seed(42)  # for reproducibility
shuffled_indices = np.random.permutation(len(combined_task))
combined_task = combined_task[shuffled_indices]

print(f"Combined REST epochs: {len(combined_rest)}")
print(f"Combined TASK epochs: {len(combined_task)}")

In [ ]:
%matplotlib qt 
#combined_task.plot(n_channels=8, title="Combined Task Epochs")

# DFA Analysis

α ≈ 0.7–1.0: Indicates long-range temporal correlations (LRTC) and near-critical dynamics

In [ ]:
# DFA ON RAW SIGNAL

In [ ]:
# Define channels of interest and fitting range
channels = ['C3', 'Cz', 'C4']
fit_range = (50, 5000)  # in samples

# Compute DFA on raw signal for both conditions
dfa_raw_rest = mf.compute_dfa_from_epochs(combined_rest, picks=channels, fit_range=fit_range)
dfa_raw_task = mf.compute_dfa_from_epochs(combined_task, picks=channels, fit_range=fit_range)

# Print results
print("\nRaw Signal DFA Results:")
print("-" * 30)
print("Channel | Rest Alpha | Task Alpha")
print("-" * 30)
for ch in channels:
    print(f"{ch:7} | {dfa_raw_rest[ch]:.3f} | {dfa_raw_task[ch]:.3f}")

# Visual comparison for channel Cz
mf.compare_conditions_dfa(
    {'Rest': combined_rest, 'Task': combined_task}, 
    channel='Cz', 
    fit_range=fit_range
)

In [ ]:
# DFA ON FREQUENCY BANDS

In [ ]:
# Define frequency bands
bands = {
    "alpha": (8, 13),
    "beta": (13, 30),
    "gamma": (30, 45)
}

# Analyze each band for each condition
for band_name, freq_range in bands.items():
    print(f"\n{band_name.upper()} BAND ({freq_range[0]}-{freq_range[1]} Hz)")
    print("-" * 50)
    
    # Compare conditions
    results = mf.compare_conditions_dfa(
        {'Rest': combined_rest, 'Task': combined_task},
        channel='Cz',
        band=freq_range,
        fit_range=(30, 1000)  # Adjusted for envelope signal
    )

In [ ]:
# DETAILED ANALYSIS FOR BETA BAND (13-30 Hz)

In [ ]:
# Analyze beta band across all channels
channels_extended = ['C3', 'Cz', 'C4', 'FC3', 'FCz', 'FC4', 'CP3', 'CPz', 'CP4']
beta_band = (13, 30)

# Compute DFA on beta envelope for all channels
dfa_beta_rest = mf.compute_dfa_from_epochs(
    combined_rest, 
    picks=channels_extended, 
    band=beta_band, 
    fit_range=(30, 1000)
)

dfa_beta_task = mf.compute_dfa_from_epochs(
    combined_task, 
    picks=channels_extended, 
    band=beta_band, 
    fit_range=(30, 1000)
)

# Create a comparison table
print("\nBeta Band DFA Results:")
print("-" * 40)
print("Channel | Rest Alpha | Task Alpha | Difference")
print("-" * 40)
for ch in channels_extended:
    diff = dfa_beta_task[ch] - dfa_beta_rest[ch]
    print(f"{ch:7} | {dfa_beta_rest[ch]:.3f} | {dfa_beta_task[ch]:.3f} | {diff:+.3f}")

# Visualize beta DFA topographically
plt.figure(figsize=(12, 5))

# Create arrays for plotting
dfa_values_rest = np.array([dfa_beta_rest[ch] for ch in channels_extended])
dfa_values_task = np.array([dfa_beta_task[ch] for ch in channels_extended])
dfa_diff = dfa_values_task - dfa_values_rest

# Extract montage info for plotting
picks = mne.pick_channels(combined_rest.info['ch_names'], channels_extended)
pos = mne.channels.layout._find_topomap_coords(combined_rest.info, picks)

# Plot the three conditions
plt.subplot(1, 3, 1)
mne.viz.plot_topomap(dfa_values_rest, pos, names=channels_extended, 
                    show_names=True, cmap='viridis', contours=0,
                    vlim=(0.5, 1.0))
plt.title('Beta DFA - Rest')

plt.subplot(1, 3, 2)
mne.viz.plot_topomap(dfa_values_task, pos, names=channels_extended, 
                    show_names=True, cmap='viridis', contours=0,
                    vlim=(0.5, 1.0))
plt.title('Beta DFA - Task')

plt.subplot(1, 3, 3)
mne.viz.plot_topomap(dfa_diff, pos, names=channels_extended, 
                    show_names=True, cmap='RdBu_r', contours=0,
                    vlim=(-0.2, 0.2))
plt.title('Task - Rest Difference')

plt.tight_layout()
plt.show()

In [ ]:
print("\n" + "="*80)
print("SUMMARY & DISCUSSION")
print("="*80)

# Calculate overall mean DFA values
mean_raw_rest = np.mean([dfa_raw_rest[ch] for ch in channels])
mean_raw_task = np.mean([dfa_raw_task[ch] for ch in channels])
mean_beta_rest = np.mean([dfa_beta_rest[ch] for ch in channels_extended])
mean_beta_task = np.mean([dfa_beta_task[ch] for ch in channels_extended])

print(f"Mean Raw DFA - Rest: {mean_raw_rest:.3f}")
print(f"Mean Raw DFA - Task: {mean_raw_task:.3f}")
print(f"Mean Beta DFA - Rest: {mean_beta_rest:.3f}")
print(f"Mean Beta DFA - Task: {mean_beta_task:.3f}")

# Discuss findings
print("\nINTERPRETATION:")
print("-" * 50)
print("DFA α values interpretation:")
print("- α < 0.5: Anti-persistent (negatively correlated)")
print("- α ≈ 0.5: White noise (uncorrelated)")
print("- 0.5 < α < 1.0: Persistent long-range correlations")
print("- α ≈ 1.0: 1/f noise (pink noise)")
print("- α > 1.0: Non-stationary, unbounded")

# Check for significant differences
if abs(mean_beta_task - mean_beta_rest) > 0.1:
    print("\nSignificant difference detected in beta band DFA between rest and task.")
    if mean_beta_task > mean_beta_rest:
        print("Task condition shows more persistent correlations (more complexity).")
    else:
        print("Rest condition shows more persistent correlations (more complexity).")
else:
    print("\nNo major differences detected in DFA scaling between rest and task.")